## 11_1 使用 ```_slots_```

Python 作为动态语言可以在程序运行过程中动态的给实例或者类添加属性，或者方法（函数）。

通过 ```__slots__``` 方法可以限制 class 实例能添加的功能。

例如：
```python
class Student(object):
    __slots__ = ('name', 'age')
```

限制了 Student 只能绑定 ```name```, ```age``` 两个属性。（只对当前类有作用，对其子类无效） 

In [8]:
class Student(object):
    __slots__ = ('name', 'age')

s = Student()
s.name = 'yui'

In [9]:
s.power = 999

AttributeError: 'Student' object has no attribute 'power'

In [ ]:
class SuperStudent(Student):
    power = 999

In [ ]:
ss = SuperStudent()

ss.mega_power = 999

当子类定义了 ```__slots__```, 其最终的 ```__slots__``` 等于自身定义的 ```__slots__``` 加上父类的 ```__slots__```。

In [11]:
class SuperStudent(Student):
    power = 999

    __slots__ = ('sad')

ss = SuperStudent()

ss.name = 999

In [12]:
ss.mega_power = 999

AttributeError: 'SuperStudent' object has no attribute 'mega_power'

## 11_2 使用 ```@property```

在 10.2 节中描述了类的访问限制相关的内容。

我们需要设置方法(set_val(), get_val())来对类的属性进行访问和修改，但是使用起来比较繁琐，需要反复调用函数。

那么能不能在限制访问后，依旧可以像直接调用属性一样访问和修改呢？ 
于是有了 ```@property```

例如：
```python
class Student(object):
    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value

    @property
    def age(self):
        return 2015 - self._birth
```

birth 设置了 @property 和 @birth.setter 可以通过属性的方式访问和修改。

age 只设置了 @property， 则只能通过属性方法访问，无法修改。相当于一个只读属性。

### Practise

请利用```@property```给一个```Screen```对象加上```width```和```height```属性，以及一个只读属性```resolution```：

In [3]:
class Screen(object):

    def __init__(self):
        self._width = 0
        self._height = 0
    
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, val):
        self._width = val

    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, val):
        self._height = val

    @property
    def resolution(self):
        return self._width * self._height
    

# 测试:
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

resolution = 786432
测试通过!


## 11.3 多重继承

通过多重继承，一个子类就可以同时获得多个父类的所有功能。

例如动物可以分为哺乳类和鸟类， 根据是否能飞行，鸟类和哺乳类又可以各自分为 2 类，

共计 $2^2 = 4$ 种。

如果进一步根据是否能当宠物，每一种子类又可以划分为 2 类。

共计 $2^3=8$ 种

可以看到类的数目呈指数级增长。

这时我们可以通过定义 能飞行的类，不能飞行的类。

通过同时继承鸟类和不能飞行，实习不能飞行的鸟类。

对于哺乳类和能否当宠物同理，这样类别的增长就是线性的。

In [4]:
class Animal(object):
    pass

# 大类:
class Mammal(Animal):
    pass

class Bird(Animal):
    pass

# 各种动物:
class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass

class Runnable(object):
    def run(self):
        print('Running...')

class Flyable(object):
    def fly(self):
        print('Flying...')


class Dog(Mammal, Runnable):
    pass

class Bat(Mammal, Flyable):
    pass

### MixIn

类的继承通常都是主线单一继承的，如果需要补充额外的功能，多重继承其它类的设计通常称之为 ```MixIn```。

为了更好地看出继承关系，我们把前面的```Runnable```和```Flyable```改为```RunnableMixIn```和```FlyableMixIn```。类似的，你还可以定义出肉食动物```CarnivorousMixIn```和植食动物```HerbivoresMixIn```，让某个动物同时拥有好几个```MixIn```：

Python自带的很多库也使用了MixIn。举个例子，Python自带了```TCPServer```和```UDPServer```这两类网络服务，而要同时服务多个用户就必须使用多进程或多线程模型，这两种模型由```ForkingMixIn```和```ThreadingMixIn```提供。通过组合，我们就可以创造出合适的服务来。

比如，编写一个多进程模式的TCP服务，定义如下：
```python
class MyTCPServer(TCPServer, ForkingMixIn):
    pass
```
编写一个多线程模式的UDP服务，定义如下：
```python
class MyUDPServer(UDPServer, ThreadingMixIn):
    pass
```

如果你打算搞一个更先进的协程模型，可以编写一个```CoroutineMixIn```：
```python
class MyTCPServer(TCPServer, CoroutineMixIn):
    pass
```
这样一来，我们不需要复杂而庞大的继承链，只要选择组合不同的类的功能，就可以快速构造出所需的子类。

## 11.4 定制类


- ```__str__()```:

- ```repr__()```:

- ```__iter__()```: 返回一个迭代器，即含有 yield 迭代函数对象，让类可以支持 ```for ... in```。

- ```__getitem__()```: 让对象可以像 list 一样通过下标访问。同样也可以支持切片和关键词访问，但是需要自行定义和实现。
对应的还有 ```__setitem__()```, 和```__delitem__()```用来执行赋值和删除操作。

- ```__getattr__()```:  当对象访问了某个属性，但是没有找到就会进入 ```__getattr__()``` 函数进行查找。如果想让 class 只返回几个特定的属性，
  我们需要在未找到对应属性的情况下抛出 ```Attribute``` 错误。
  例如：
  ```python
  class Student(object):
    def __getattr__(self, attr):
        if attr=='age':
            return lambda: 25
        raise AttributeError('\'Student\' object has no attribute \'%s\'' % attr)
  ```

REST API 中，获取链接接口的 API 是非常多的。要给每个链接写一个方法显然不现实，这时可以通过动态的 ```__getattr__()``` 方法实现一个链式调用。

具体代码如下：

In [1]:
class Chain(object):
    def __init__(self, path=''):
        self._path = path

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__


In [6]:
t = Chain().status.user.timeline.list
t

/status/user/timeline/list

In [5]:
type(t)

__main__.Chain

- ```__call__()```: 任何类，只需要定义一个__call__()方法，就可以直接对实例进行调用。

示例：

In [7]:
class Student(object):
    def __init__(self, name):
        self.name = name

    def __call__(self):
        print('My name is %s.' % self.name)

s = Student('yuukilight')
s() # self参数不要传入

My name is yuukilight.


通过```callable()```函数，我们就可以判断一个对象是否是“可调用”对象。

## 11.5 使用枚举类

In [8]:
from enum import Enum

Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [18]:
Month.Mar

<Month.Mar: 3>

In [16]:
Month.Mar.value

3

In [14]:
for name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Jul => Month.Jul , 7
Aug => Month.Aug , 8
Sep => Month.Sep , 9
Oct => Month.Oct , 10
Nov => Month.Nov , 11
Dec => Month.Dec , 12


```value```属性则是自动赋给成员的int常量，默认从1开始计数。

如果需要更精确地控制枚举类型，可以从Enum派生出自定义类：

In [17]:
from enum import Enum, unique

@unique
class Weekday(Enum):
    Sun = 0 # Sun的value被设定为0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6


```Enum```可以把一组相关常量定义在一个class中，且class不可变，而且成员可以直接比较。

### Practise
把```Student```的```gender``属性改造为枚举类型，可以避免使用字符串：

In [19]:
from enum import Enum, unique

class Gender(Enum):
    Male = 0
    Female = 1

class Student(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender

# 测试:
bart = Student('Bart', Gender.Male)
if bart.gender == Gender.Male:
    print('测试通过!')
else:
    print('测试失败!')


测试通过!
